In [1]:
%matplotlib
#%matplotlib inline
import numpy as np
from math import comb
import matplotlib.pyplot as plt
from numba import njit, jit
import scipy
#import timeit
#from mpl_toolkits import mplot3d

Using matplotlib backend: <object object at 0x0000017D724AA570>


In [2]:
nome = 'Ge.xyz'
f = open(nome)
data = f.readlines() #Retorna uma lista contendo as linhas do arquivo
data = [line.strip().split() for line in data] #Remove os espaços e isola as coordenadas
data

[['216'],
 ['100', '0', '0'],
 ['Ge', '0.0001519560510179', '0.01977331053476', '-0.01993796069407'],
 ['Ge', '-0.01942048447972', '2.805893848131', '2.816428497902'],
 ['Ge', '2.853087214211', '2.775248718765', '-0.01769086580866'],
 ['Ge', '2.833823954538', '0.01975747415924', '2.780607937414'],
 ['Ge', '4.178109282151', '1.394845085369', '4.2918255843'],
 ['Ge', '1.474720793264', '1.400055963729', '1.352129067726'],
 ['Ge', '1.399292158155', '4.272402393784', '4.210283849562'],
 ['Ge', '4.198437666375', '4.241669966627', '1.414977656161'],
 ['Ge', '0.04540562706355', '0.06376307148138', '5.54416911922'],
 ['Ge', '0.04897898055329', '2.888442801728', '8.482100897031'],
 ['Ge', '2.86039811833', '2.821666338714', '5.640049899695'],
 ['Ge', '2.808462840849', '-0.0302073222241', '8.492837931052'],
 ['Ge', '4.269546980003', '1.405378618015', '9.856453035567'],
 ['Ge', '1.355881052191', '1.468375697148', '7.066875592878'],
 ['Ge', '1.46800265354', '4.188099022277', '9.911629579895'],
 ['Ge

In [3]:
num_atoms = int(data[0][0]) #Retorna como int o primeiro elemento da primeira lista/linha = número de átomos
dims = 3 #Número de dimensões
coords = np.zeros((num_atoms,dims)) #Linha,coluna
#coords_2 = coords.copy()

In [4]:
#%%timeit
for i in range(num_atoms):
    x = float(data[i+2][1]) #+2: átomos começam na terceira linha do arquivo
    y = float(data[i+2][2]) 
    z = float(data[i+2][3])
    coords[i] = x,y,z

In [5]:
#Testes dos dados

#print(data[0])
#print(data[0][0])
#print(data[1])
#print(data[2])
#print(data[1][1])
print(len(coords))
print(coords)

216
[[ 1.51956051e-04  1.97733105e-02 -1.99379607e-02]
 [-1.94204845e-02  2.80589385e+00  2.81642850e+00]
 [ 2.85308721e+00  2.77524872e+00 -1.76908658e-02]
 [ 2.83382395e+00  1.97574742e-02  2.78060794e+00]
 [ 4.17810928e+00  1.39484509e+00  4.29182558e+00]
 [ 1.47472079e+00  1.40005596e+00  1.35212907e+00]
 [ 1.39929216e+00  4.27240239e+00  4.21028385e+00]
 [ 4.19843767e+00  4.24166997e+00  1.41497766e+00]
 [ 4.54056271e-02  6.37630715e-02  5.54416912e+00]
 [ 4.89789806e-02  2.88844280e+00  8.48210090e+00]
 [ 2.86039812e+00  2.82166634e+00  5.64004990e+00]
 [ 2.80846284e+00 -3.02073222e-02  8.49283793e+00]
 [ 4.26954698e+00  1.40537862e+00  9.85645304e+00]
 [ 1.35588105e+00  1.46837570e+00  7.06687559e+00]
 [ 1.46800265e+00  4.18809902e+00  9.91162958e+00]
 [ 4.26483870e+00  4.18534356e+00  7.09552201e+00]
 [-3.50205326e-02  1.13657070e-02  1.13850688e+01]
 [-5.46639355e-02  2.82221447e+00  1.41954695e+01]
 [ 2.79581036e+00  2.82861497e+00  1.14162826e+01]
 [ 2.84966997e+00  4.930867

In [6]:
#Introduzindo condições de contorno periódicas
a_lat = 3 * 5.6575398445 #Aresta do cubo

#Distâncias entre vizinhos
d_1 = 2.45
d_2 = 4.0
d_3 = 4.69
d_4 = 5.66
delta = 0.2

In [7]:
@njit
def calcular_distancias(coordenadas, aresta):
    lista_i = []
    lista_j = []
    distancias = []
    #pares = []
    for i in range(len(coordenadas)):
        x_0, y_0, z_0 = coords[i]
        for j in range(len(coordenadas)):
            if j in lista_i and i in lista_j:
                continue
            for k in [-1,0,1]:
                for l in [-1,0,1]:
                    for m in [-1,0,1]:
                        shift = aresta * np.asarray([k,l,m]) #Shift -> CCP -> fator vezes -1, 0 ou 1 -> Diferença p + ou p -
                        x, y, z = coords[j] + shift #Coordenadas do átomo j corrigidas com o shift
                        rr = np.asarray([x-x_0, y-y_0, z-z_0]) #Vetor rr = dif. em posição do átomo i com átomo j corrigido
                        dist = np.linalg.norm(rr) #Norma do vetor rr (distância)
                        lista_i.append(i)
                        lista_j.append(j)
                        distancias.append(dist)
                        #pares.append((i,j))
    return distancias

In [8]:
dist = calcular_distancias(coords, a_lat)

In [9]:
%%timeit
dist = calcular_distancias(coords, a_lat)
dv1 = []
dv2 = []
dv3 = []
dv4 = []
for i in range(len(dist)):
    if dist[i] <= (d_1 + delta) and dist[i] > (d_1 - delta): # primeiros vizinhos
        dv1.append(dist)
    elif dist[i] <= (d_2 + delta) and dist[i] > (d_2 - delta): # segundos vizinhos
        dv2.append(dist)
    elif dist[i] <= (d_3 + delta) and dist[i] > (d_3 - delta): # terceiros vizinhos
        dv3.append(dist)
    elif dist[i] <= (d_4 + delta) and dist[i] > (d_4 - delta): # quartos vizinhos
        dv4.append(dist)

8.58 s ± 269 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
print(dv1)

NameError: name 'dv1' is not defined

In [ ]:
%%timeit
#Distâncias entre vizinhos não são iguais
d_1 = 2.45
d_2 = 4.0
d_3 = 4.69
d_4 = 5.66
delta = 0.2 #"Incerteza"?

#Lista com distâncias
distances = []
dist_v1 = []
dist_v2 = []
dist_v3 = []
dist_v4 = []

#Dicionários com vizinhos
First_N = {}
Second_N = {}
Third_N = {}
Fourth_N = {}

#Para cada átomo i:
for i in range(num_atoms):
    #Definir os vizinhos
    viz1 = []
    viz2 = []
    viz3 = []
    viz4 = []
    #Recuperar coordenadas do átomo em questão
    x_0,y_0,z_0 = coords[i]
    #Para cada átomo j por átomo i:
    for j in range(num_atoms):
        #Checar se átomos já foram classificados como vizinhos de outros:
        if First_N.get(j) is not None and i in First_N[j]:
            viz1.append(j)
        elif Second_N.get(j) is not None and i in Second_N[j]:
            viz2.append(j)
        elif Third_N.get(j) is not None and i in Third_N[j]:
            viz3.append(j)
        elif Fourth_N.get(j) is not None and i in Fourth_N[j]:
            viz4.append(j)
        #Se o átomo não foi classificado ainda:
        else: 
            #Definir o fator do shift -> CCP
            for k in [-1,0,1]:
                for l in [-1,0,1]:
                    for m in [-1,0,1]:
                        shift = a_lat * np.asarray([k,l,m]) #Shift -> CCP -> fator vezes -1, 0 ou 1 -> Diferença p + ou p -
                        x, y, z = coords[j] + shift #Coordenadas do átomo j corrigidas com o shift
                        rr = np.asarray([x-x_0, y-y_0, z-z_0]) #Vetor rr = dif. em posição do átomo i com átomo j corrigido
                        dist = np.linalg.norm(rr) #Norma do vetor rr (distância)
                        distances.append(dist) #Adicionar distância de i a j na lista de distâncias
                        #Condições para determinar se são vizinhos:
                        if dist <= (d_1 + delta) and dist > (d_1 - delta): # primeiros vizinhos
                            viz1.append(j)
                            dist_v1.append(dist)
                        elif dist <= (d_2 + delta) and dist > (d_2 - delta): # segundos vizinhos
                            viz2.append(j)
                            dist_v2.append(dist)
                        elif dist <= (d_3 + delta) and dist > (d_3 - delta): # terceiros vizinhos
                            viz3.append(j)
                            dist_v3.append(dist)
                        elif dist <= (d_4 + delta) and dist > (d_4 - delta): # quartos vizinhos
                            viz4.append(j)
                            dist_v4.append(dist)
    First_N[i] = viz1
    Second_N[i]= viz2
    Third_N[i] = viz3
    Fourth_N[i]= viz4

dist_t = dist_v1 + dist_v2 + dist_v3 + dist_v4
len(distances)

In [ ]:
First_N

In [ ]:
Second_N

In [ ]:
Third_N

In [ ]:
Fourth_N

In [ ]:
dist_v1

In [ ]:
#Teste dos valores de distância

print(len(distances))
print(max(distances))
print(min(distances))
print(len(dist_t))

In [ ]:
#Histograma com frequências em y
plt.hist(dist_t, bins=300)
plt.xlabel('Distâncias interatômicas')
plt.ylabel('Frequência')
plt.show()

In [ ]:
#Histograma com probabilidades em y
plt.hist(dist_t, bins=300, color='red', density=True)
plt.xlabel('Distâncias interatômicas')
plt.ylabel('Probabilidade')
plt.show()

In [ ]:
#Histograma teste das distâncias radiais --> sem condições de contorno

plt.hist(dist_2, bins=150)
plt.xlabel('Distâncias interatômicas')
plt.ylabel('Frequência')
plt.show()

In [ ]:
# Determinando propriedades da primeira camada de coordenação
#First_N

In [ ]:
#Teste das coordenadas

#print(coords)
#print(coords[0,:])
#print(coords[0])
#print(coords[1])

In [ ]:
print('Primeiros vizinhos')
for key in First_N: #Para cada átomo key
    i = key
    r_i = coords[i,:] #Coordenadas de i
    print('Átomo', i)
    for element in First_N[i]: #Para cada átomo vizinho de i
        j=element
        #print(j)
        r_j = coords[j,:] #Coordenadas do vizinho j
        rr = r_j - r_i #Vetor dif. posição entre i e j
        dist = np.linalg.norm(rr) #Módulo do vetor = distância
        #print(i, dist) #Mostrar átomo i e distância até o átomo j
        print('Vizinho: átomo', j, ' ', 'Distância =', dist) #Mostrar vizinho j e distância até ele